In [1]:
import pandas as pd
import  os
import numpy as np
import psycopg2
import glob


In [3]:
df = pd.read_excel("filename.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# replace spaces in column names with underscores or anything else
df.columns = [x.lower().replace("#","number").replace(" ","_").replace("%","").replace(".","") for x in df.columns]
df.columns

Index(['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate',
       'unitprice', 'customerid', 'country'],
      dtype='object')

In [5]:
df.dtypes

invoiceno              object
stockcode              object
description            object
quantity                int64
invoicedate    datetime64[ns]
unitprice             float64
customerid            float64
country                object
dtype: object

In [6]:
#change datatypes to acceptable ones in the database
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'varchar',
    'datetime64[ns]': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'varchar',
 'datetime64[ns]': 'timestamp',
 'bool': 'varchar'}

In [7]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'invoiceno varchar, stockcode varchar, description varchar, quantity varchar, invoicedate timestamp, unitprice float, customerid float, country varchar'

In [11]:
#database connection setup
conn_string = "host= localhost \
                dbname='postgres' \
                    user='postgres' password='?????' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [13]:
cursor.execute("drop table if exists commerce_data;")

In [14]:
cursor.execute("create table commerce_data \
    (invoiceno varchar, stockcode varchar, description varchar, quantity varchar, invoicedate timestamp, unitprice float, customerid float, country varchar)")

In [16]:
#insert values to tables
#save df to csv
df.to_csv('commerce_data', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("commerce_data",)
print('file opened in memory')

file opened in memory


In [17]:
SQL_STATEMENT = """
COPY commerce_data FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [18]:
cursor.execute("grant select on table commerce_data to public")
conn.commit()

cursor.close()
print('table commerce_data imported successfully')

table commerce_data imported successfully
